In [ ]:
from google.colab import drive # Menyambungkan ke google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Melihat nama file dari folder data set
!ls "/content/drive/MyDrive/PSD/Data Set"

10_ANG.wav  12_SAD.wav	15_NEU.wav  1_Marah.wav  4_FEA.wav  7_DIS.wav
10_DIS.wav  13_ANG.wav	15_SAD.wav  1_NEU.wav	 4_HAP.wav  7_FEA.wav
10_FEA.wav  13_DIS.wav	16_ANG.wav  1_SAD.wav	 4_NEU.wav  7_HAP.wav
10_HAP.wav  13_FEA.wav	16_DIS.wav  2_DIS.wav	 4_SAD.wav  7_NEU.wav
10_NEU.wav  13_HAP.wav	16_FEA.wav  2_FEA.wav	 5_ANG.wav  7_SAD.wav
10_SAD.wav  13_NEU.wav	16_HAP.wav  2_HAP.wav	 5_DIS.wav  8_ANG.wav
11_ANG.wav  13_SAD.wav	16_NEU.wav  2_Marah.wav  5_FEA.wav  8_DIS.wav
11_DIS.wav  14_ANG.wav	16_SAD.wav  2_NEU.wav	 5_HAP.wav  8_FEA.wav
11_FEA.wav  14_DIS.wav	17_ANG.wav  2_SAD.wav	 5_NEU.wav  8_HAP.wav
11_HAP.wav  14_FEA.wav	17_DIS.wav  3_DIS.wav	 5_SAD.wav  8_NEU.wav
11_NEU.wav  14_HAP.wav	17_FEA.wav  3_FEA.wav	 6_ANG.wav  8_SAD.wav
11_SAD.wav  14_NEU.wav	17_HAP.wav  3_HAP.wav	 6_DIS.wav  9_ANG.wav
12_ANG.wav  14_SAD.wav	17_NEU.wav  3_Marah.wav  6_FEA.wav  9_DIS.wav
12_DIS.wav  15_ANG.wav	17_SAD.wav  3_NEU.wav	 6_HAP.wav  9_FEA.wav
12_FEA.wav  15_DIS.wav	1_DIS.wav   3_SAD.wav	 6_NEU.wav 

# Normalisasi

In [ ]:
import os
import librosa
import numpy as np
import soundfile as sf
from IPython.display import Audio, display

Dataset= "/content/drive/MyDrive/PSD/Data Set" # Diambil dari folder Data Set
Normalisasi= "/content/drive/MyDrive/PSD/Normalized" # Outputnya di simpan ni folder Normalized


os.makedirs(Normalisasi, exist_ok=True)

# Fungsi normalisasi
def normalize_audio(x):
    return x / np.max(np.abs(x))

# Ambil list file .wav
wav_files = [f for f in os.listdir(Dataset) if f.endswith(".wav")]

print("Jumlah file WAV:", len(wav_files))

# Loop normalisasi + tampilkan output
for file in wav_files:
    # path input & output
    path_in = os.path.join(Dataset, file)
    path_out = os.path.join(Normalisasi, file)

    # memuat file audio
    y, sr = librosa.load(path_in, sr=None)

    # normalisasi
    y_norm = normalize_audio(y)

    # simpan hasil
    sf.write(path_out, y_norm, sr)

    print("Normalisasi:", file)
    print("Sebelum Normalisasi:")
    display(Audio(path_in))

    print("Sesudah Normalisasi:")
    display(Audio(path_out))

**Menampilkan Wavefrom dari hasil Normalisasi**

Catatan: Fungsi normalisasi di sini bertujuan untuk "menyamakan level" audio sehingga amplitudo suara menjadi lebih konsisten antar file.
Hal ini membantu ketika kita ingin membandingkan atau memproses audio lebih lanjut.


In [ ]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display

DATASET = "/content/drive/MyDrive/PSD/Data Set" # Diambil dari folder Dataset
NORMALIZED = "/content/drive/MyDrive/PSD/Normalized" # Outpunya di simpan di folder Normalized

wav_files = [f for f in os.listdir(NORMALIZED) if f.endswith(".wav")]

for file in wav_files:
    path_asli = os.path.join(DATASET, file)
    path_norm = os.path.join(NORMALIZED, file)

    # memuat audio
    y_asli, sr = librosa.load(path_asli, sr=None)
    y_norm, sr = librosa.load(path_norm, sr=None)

    time_asli = np.arange(len(y_asli)) / sr
    time_norm = np.arange(len(y_norm)) / sr

    print(f"\n=== {file} ===")
    print("Audio Asli:")
    display(Audio(path_asli))

    print("Audio Normalisasi:")
    display(Audio(path_norm))

    # Plot perbandingan waveform asli dan normalisasi
    plt.figure(figsize=(14,5))
    plt.plot(time_norm, y_norm, label="Normalisasi", alpha=0.7, color="#FFE52A")
    plt.plot(time_asli, y_asli, label="Asli", alpha=0.7,color="#5A9CB5")
    plt.title(f"Perbandingan Waveform: {file}")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Amplitude")
    plt.legend()
    plt.grid(True)
    plt.show()


# LPF

Low-Pass Filter (LPF) adalah proses penyaringan sinyal audio yang berfungsi untuk melewatkan frekuensi rendah dan mengurangi frekuensi tinggi di atas 3400 Hz. Filter ini digunakan untuk menghilangkan noise dan membuat suara menjadi lebih halus serta jelas. Dengan cutoff 3400 Hz, sinyal yang dipertahankan adalah frekuensi utama suara manusia sehingga audio lebih stabil dan mudah dianalisis.

In [ ]:
import os
import librosa
import numpy as np
import soundfile as sf
from scipy.signal import butter, filtfilt
from IPython.display import Audio, display
import matplotlib.pyplot as plt

SRC = "/content/drive/MyDrive/PSD/Normalized" # Mengambil hasil folder Normalized
DST_LPF = "/content/drive/MyDrive/PSD/LPF" # Outputnya di simpan di folder LPF
os.makedirs(DST_LPF, exist_ok=True)

# Fungsi Low-Pass Filter
def low_pass_filter(signal, sr, cutoff=3400, order=5):
    nyq = 0.5 * sr
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered = filtfilt(b, a, signal)
    return filtered

wav_files = [f for f in os.listdir(SRC) if f.endswith(".wav")]

for file in wav_files:
    path_in = os.path.join(SRC, file)
    y, sr = librosa.load(path_in, sr=None)

    y_lpf = low_pass_filter(y, sr)
    path_out = os.path.join(DST_LPF, file)
    sf.write(path_out, y_lpf, sr)

    print("\n=== LOW-PASS FILTER:", file, "===")

    print("Sebelum LPF:") # Sebelum
    display(Audio(path_in))
    print("Sesudah LPF:") #Sesudah
    display(Audio(path_out))

    # Plot perbandingan waveform
    time = np.arange(len(y)) / sr
    time_lpf = np.arange(len(y_lpf)) / sr

    plt.figure(figsize=(14, 5))
    plt.plot(time, y, label='Original', alpha=0.7)
    plt.plot(time_lpf, y_lpf, label='LPF', alpha=0.7)
    plt.title(f"Waveform: {file}")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Amplitude")
    plt.legend()
    plt.show()


# BPF

Band-Pass Filter (BPF) adalah proses penyaringan sinyal audio yang hanya melewatkan frekuensi dalam rentang 300–3400 Hz dan menghilangkan frekuensi di bawah maupun di atas rentang tersebut. Filter ini digunakan untuk mempertahankan frekuensi utama suara manusia serta mengurangi noise rendah dan tinggi. Dengan BPF, sinyal audio menjadi lebih fokus, bersih, dan lebih mudah dianalisis.

In [ ]:
import os
import librosa
import numpy as np
import soundfile as sf
from scipy.signal import butter, filtfilt
from IPython.display import Audio, display
import matplotlib.pyplot as plt

SRC = "/content/drive/MyDrive/PSD/Normalized" # Mengambil dari folder Normalized
DST_BPF = "/content/drive/MyDrive/PSD/BPF" # Outputnya disimpan dalam folder BPF
os.makedirs(DST_BPF, exist_ok=True)

# Fungsi Bandpass Filter (300–3400 Hz)
def bandpass_filter(signal, sr, lowcut=300, highcut=3400, order=5):
    nyq = 0.5 * sr
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    filtered = filtfilt(b, a, signal)
    return filtered

# Ambil semua file wav
wav_files = [f for f in os.listdir(SRC) if f.endswith(".wav")]

for file in wav_files:
    path_in = os.path.join(SRC, file)
    y, sr = librosa.load(path_in, sr=None)

    # BPF
    y_bpf = bandpass_filter(y, sr)

    # Simpan hasil BPF
    path_out = os.path.join(DST_BPF, file)
    sf.write(path_out, y_bpf, sr)

    print("\n=== BAND-PASS FILTER:", file, "===")

    #--Audio sebelum BPF--
    print("Sebelum BPF:")
    display(Audio(path_in))

    # --Audio sesudah BPF--
    print("Sesudah BPF:")
    display(Audio(path_out))

    # Membuat sumbu waktu (detik)
    time = np.arange(len(y)) / sr
    time_bpf = np.arange(len(y_bpf)) / sr

    # Waveform plot
    plt.figure(figsize=(14, 5))
    plt.plot(time, y, label='Original', alpha=0.7)
    plt.plot(time_bpf, y_bpf, label='BPF (300–4000 Hz)', alpha=0.7)
    plt.title(f"Waveform Band-Pass Filter: {file}")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Amplitude")
    plt.legend()
    plt.show()


# SPECTOGRAM

In [ ]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt

SRC_Normal = "/content/drive/MyDrive/PSD/Normalized"
SRC_LPF = "/content/drive/MyDrive/PSD/LPF"
SRC_BPF = "/content/drive/MyDrive/PSD/BPF"

DST_MEL_Normal ="/content/drive/MyDrive/PSD/Mel_Normal"
DST_MEL_LPF = "/content/drive/MyDrive/PSD/Mel_LPF"
DST_MEL_BPF = "/content/drive/MyDrive/PSD/Mel_BPF"

os.makedirs(DST_MEL_Normal, exist_ok=True)
os.makedirs(DST_MEL_LPF, exist_ok=True)
os.makedirs(DST_MEL_BPF, exist_ok=True)

def make_mel(path, sr_target=16000):
    y, sr = librosa.load(path, sr=sr_target)
    y_trim, _ = librosa.effects.trim(y)

    mel = librosa.feature.melspectrogram(
        y=y_trim,
        sr=sr_target,
        n_fft=2048,
        hop_length=512,
        n_mels=128
    )
    mel_db = librosa.power_to_db(mel, ref=np.max)

    return mel_db

def process_folder(src, dst):
    files = [f for f in os.listdir(src) if f.endswith(".wav")]
    for file in files:
        mel = make_mel(os.path.join(src, file))
        np.save(os.path.join(dst, file.replace(".wav", ".npy")), mel)
        print("Done:", file)

print("=== Proses Normal → Mel-Spectrogram ===")
process_folder(SRC_Normal, DST_MEL_Normal)

print("=== Proses LPF → Mel-Spectrogram ===")
process_folder(SRC_LPF, DST_MEL_LPF)

print("=== Proses BPF → Mel-Spectrogram ===")
process_folder(SRC_BPF, DST_MEL_BPF)

print("Selesai!")


# MEL SPECTOGRAM NORMAL

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Folder Mel-Spectrogram
FOLDER = "/content/drive/MyDrive/PSD/Mel_Normal"

# Ambil semua file .npy
mel_files = [f for f in os.listdir(FOLDER) if f.endswith(".npy")]

print("Jumlah file:", len(mel_files))

# Loop untuk menampilkan semua spectrogram
for file in mel_files:
    path = os.path.join(FOLDER, file)

    # Load mel-spectrogram
    mel = np.load(path)

    mel_bins = mel.shape[0]
    time_frames = mel.shape[1]

    print(f"\n=== {file} ===")
    print(f"Mel bins: {mel_bins}")
    print(f"Time frames: {time_frames}")

    # Plot
    plt.figure(figsize=(8, 4))
    plt.imshow(mel, aspect='auto', origin='lower')
    plt.colorbar()
    plt.title(f"Mel-Spectrogram: {file}")
    plt.xlabel(f"Frame Waktu ({time_frames})")
    plt.ylabel(f"Frekuensi ({mel_bins} Mel bins)")
    plt.show()

# MEL SPECTOGRAM LPF

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Folder Mel-Spectrogram
FOLDER = "/content/drive/MyDrive/PSD/Mel_LPF"

# Ambil semua file .npy
mel_files = [f for f in os.listdir(FOLDER) if f.endswith(".npy")]

print("Jumlah file:", len(mel_files))

# Loop untuk menampilkan semua spectrogram
for file in mel_files:
    path = os.path.join(FOLDER, file)

    # Load mel-spectrogram
    mel = np.load(path)

    mel_bins = mel.shape[0]
    time_frames = mel.shape[1]

    print(f"\n=== {file} ===")
    print(f"Mel bins: {mel_bins}")
    print(f"Time frames: {time_frames}")

    # Plot
    plt.figure(figsize=(8, 4))
    plt.imshow(mel, aspect='auto', origin='lower')
    plt.colorbar()
    plt.title(f"Mel-Spectrogram: {file}")
    plt.xlabel(f"Frame Waktu ({time_frames})")
    plt.ylabel(f"Frekuensi ({mel_bins} Mel bins)")
    plt.show()

# MEL SPECTOGRAM BPF

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Folder Mel-Spectrogram
FOLDER = "/content/drive/MyDrive/PSD/Mel_BPF"

# Ambil semua file .npy
mel_files = [f for f in os.listdir(FOLDER) if f.endswith(".npy")]

print("Jumlah file:", len(mel_files))

# Loop untuk menampilkan semua spectrogram
for file in mel_files:
    path = os.path.join(FOLDER, file)

    # Load mel-spectrogram
    mel = np.load(path)

    mel_bins = mel.shape[0]
    time_frames = mel.shape[1]

    print(f"\n=== {file} ===")
    print(f"Mel bins: {mel_bins}")
    print(f"Time frames: {time_frames}")

    # Plot
    plt.figure(figsize=(8, 4))
    plt.imshow(mel, aspect='auto', origin='lower')
    plt.colorbar()
    plt.title(f"Mel-Spectrogram: {file}")
    plt.xlabel(f"Frame Waktu ({time_frames})")
    plt.ylabel(f"Frekuensi ({mel_bins} Mel bins)")
    plt.show()

# CNN

Sebelum dilakukan pemodelan, dilakukan persiapan data audio.

Standardisasi ukuran audio : Data audio yang memiliki durasi terlalu pendek akan diberi padding & Jika terlalu panjang akan di-crop.


In [ ]:
import numpy as np
import os

def pad_spectrogram(mel, max_len):

    if mel.shape[1] < max_len:
        pad_width = max_len - mel.shape[1]
        mel = np.pad(mel, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mel = mel[:, :max_len]
    return mel

def load_mel(folder, max_len): #memuat mel-spectogram

    X, y = [], []
    # mengurutkan nama file.
    files = sorted([f for f in os.listdir(folder) if f.endswith(".npy")])

    for file in files:
        path = os.path.join(folder, file)
        mel = np.load(path)

        # pangkas hingga ukuran seragam
        mel = pad_spectrogram(mel, max_len=max_len)

        # Tambahkan dimensi channel (untuk masukan CNN)
        mel = np.expand_dims(mel, axis=-1)

        # Ekstrak label dari nama file (e.g., '1_Happy.npy' -> 'Happy')
        label = file.split("_")[1].replace(".npy", "")

        X.append(mel)
        y.append(label)

    return np.array(X), np.array(y)

print("Fungsi bantu `pad_spectrogram` dan `load_mel` berhasil didefinisikan")

Helper functions `pad_spectrogram` and `load_mel` defined successfully.


Pemuatan Data Multi-Set yang Terstandardisasi

In [ ]:
import numpy as np
import os

# definisi folder paths
DST_MEL_Normal = "/content/drive/MyDrive/PSD/Mel_Normal"
DST_MEL_LPF = "/content/drive/MyDrive/PSD/Mel_LPF"
DST_MEL_BPF = "/content/drive/MyDrive/PSD/Mel_BPF"

# tentukan panjang maksimum di seluruh dataset untuk padding yang konsisten.
all_folders = [DST_MEL_Normal, DST_MEL_LPF, DST_MEL_BPF]
max_len_overall = 0

for folder in all_folders:
    files = [f for f in os.listdir(folder) if f.endswith(".npy")]
    for file in files:
        mel = np.load(os.path.join(folder, file))
        if mel.shape[1] > max_len_overall:
            max_len_overall = mel.shape[1]

print(f"Jangka waktu maksimum keseluruhan yang telah ditentukan: {max_len_overall}")

# muat data menggunakan max_len yang telah ditentukan.
X_normal, y_normal_labels = load_mel(DST_MEL_Normal, max_len=max_len_overall)
X_lpf, y_lpf_labels = load_mel(DST_MEL_LPF, max_len=max_len_overall)
X_bpf, y_bpf_labels = load_mel(DST_MEL_BPF, max_len=max_len_overall)

print("\nMel-Spectrograms berhasil dimuat")
print(f"Shape of X_normal: {X_normal.shape}")
print(f"Shape of y_normal_labels: {y_normal_labels.shape}")
print(f"Shape of X_lpf: {X_lpf.shape}")
print(f"Shape of y_lpf_labels: {y_lpf_labels.shape}")
print(f"Shape of X_bpf: {X_bpf.shape}")
print(f"Shape of y_bpf_labels: {y_bpf_labels.shape}")

Determined overall maximum time frame length: 101

Mel-Spectrograms loaded successfully.
Shape of X_normal: (102, 128, 101, 1)
Shape of y_normal_labels: (102,)
Shape of X_lpf: (102, 128, 101, 1)
Shape of y_lpf_labels: (102,)
Shape of X_bpf: (102, 128, 101, 1)
Shape of y_bpf_labels: (102,)


Kemudian kita ubah label kelas kategori (berupa string, seperti 'Happy', 'Sad', 'Angry') menjadi format numerik (integer) menggunakan teknik yang disebut Label Encoding.

In [ ]:
import numpy as np

# Min-max normalization function
def min_max_normalize(X):
    min_val = np.min(X[X != 0]) if np.any(X != 0) else 0 # Hindari nilai minimum yang seluruhnya nol jika padding seluruhnya nol
    max_val = np.max(X)

    if max_val == min_val: # Hindari pembagian oleh nol jika semua nilai sama
        return np.zeros_like(X)

    # terapkan min-max scaling
    X_normalized = (X - min_val) / (max_val - min_val)

    # nilai-nilai berada dalam rentang [0, 1] setelah normalisasi
    X_normalized = np.clip(X_normalized, 0, 1)

    return X_normalized

# Terapkan normalisasi min-max pada setiap dataset
X_normal_norm = min_max_normalize(X_normal)
X_lpf_norm = min_max_normalize(X_lpf)
X_bpf_norm = min_max_normalize(X_bpf)

print("Normalisasi min-max berhasil diterapkan pada X_normal, X_lpf, dan X_bpf.")
print(f"Shape of X_normal_norm: {X_normal_norm.shape}")
print(f"Min value of X_normal_norm: {np.min(X_normal_norm)}")
print(f"Max value of X_normal_norm: {np.max(X_normal_norm)}")

Min-max normalization applied to X_normal, X_lpf, and X_bpf successfully.
Shape of X_normal_norm: (102, 128, 101, 1)
Min value of X_normal_norm: 0.0
Max value of X_normal_norm: 1.0


Setelah dilakukan labelling, setiap set data dibagi (Normal, LPF, dan BPF) menjadi subset pelatihan (training) dan pengujian (testing) secara terpisah dan konsisten.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Inisialisasi LabelEncoder
le = LabelEncoder()

# Sesuaikan dan ubah label untuk setiap dataset
y_normal_enc = le.fit_transform(y_normal_labels)
y_lpf_enc = le.fit_transform(y_lpf_labels)
y_bpf_enc = le.fit_transform(y_bpf_labels)

print("Penerapan enkoding label pada semua dataset telah berhasil.")
print(f"Unique labels (original): {le.classes_}")
print(f"Encoded labels for y_normal (first 5): {y_normal_enc[:5]}")
print(f"Encoded labels for y_lpf (first 5): {y_lpf_enc[:5]}")
print(f"Encoded labels for y_bpf (first 5): {y_bpf_enc[:5]}")

Label encoding applied to all datasets successfully.
Unique labels (original): ['ANG' 'DIS' 'FEA' 'HAP' 'Marah' 'NEU' 'SAD']
Encoded labels for y_normal (first 5): [0 1 2 3 5]
Encoded labels for y_lpf (first 5): [0 1 2 3 5]
Encoded labels for y_bpf (first 5): [0 1 2 3 5]


In [ ]:
from sklearn.model_selection import train_test_split

# Split Normal dataset
X_normal_train, X_normal_test, y_normal_train, y_normal_test = train_test_split(
    X_normal_norm, y_normal_enc,
    test_size=0.35,
    random_state=42,
    stratify=y_normal_enc
)

# Split LPF dataset
X_lpf_train, X_lpf_test, y_lpf_train, y_lpf_test = train_test_split(
    X_lpf_norm, y_lpf_enc,
    test_size=0.35,
    random_state=42,
    stratify=y_lpf_enc
)

# Split BPF dataset
X_bpf_train, X_bpf_test, y_bpf_train, y_bpf_test = train_test_split(
    X_bpf_norm, y_bpf_enc,
    test_size=0.35,
    random_state=42,
    stratify=y_bpf_enc
)

print("Data set berhasil dibagi menjadi set pelatihan dan set pengujian.")
print(f"Normal - Train shapes: {X_normal_train.shape}, {y_normal_train.shape}")
print(f"Normal - Test shapes: {X_normal_test.shape}, {y_normal_test.shape}")
print(f"LPF - Train shapes: {X_lpf_train.shape}, {y_lpf_train.shape}")
print(f"LPF - Test shapes: {X_lpf_test.shape}, {y_lpf_test.shape}")
print(f"BPF - Train shapes: {X_bpf_train.shape}, {y_bpf_train.shape}")
print(f"BPF - Test shapes: {X_bpf_test.shape}, {y_bpf_test.shape}")

Datasets split into training and testing sets successfully.
Normal - Train shapes: (66, 128, 101, 1), (66,)
Normal - Test shapes: (36, 128, 101, 1), (36,)
LPF - Train shapes: (66, 128, 101, 1), (66,)
LPF - Test shapes: (36, 128, 101, 1), (36,)
BPF - Train shapes: (66, 128, 101, 1), (66,)
BPF - Test shapes: (36, 128, 101, 1), (36,)


Membangun dan mengkompilasi arsitektur (Convolutional Neural Network / CNN) yang dirancang untuk tugas klasifikasi emosi atau suara berdasarkan Mel-spectrogram.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

print("Fungsi pembangun model CNN `build_cnn_model` telah berhasil didefinisikan")


CNN model building function `build_cnn_model` defined successfully.


Melatih dan mengevaluasi model Convolutional Neural Network (CNN) secara spesifik menggunakan dataset Normal (tanpa filter LPF/BPF).

In [ ]:
input_shape = X_normal_train.shape[1:]
num_classes = len(np.unique(y_normal_enc))

print(f"Input shape for CNN: {input_shape}")
print(f"Number of classes: {num_classes}")

# Training dan Evaluasi untuk data Normal
print("\nTraining dan Evaluasi Model untuk data normal")

normal_model = build_cnn_model(input_shape, num_classes)
normal_model.summary()

history_normal = normal_model.fit(
    X_normal_train, y_normal_train,
    epochs=50,
    batch_size=2,
    validation_data=(X_normal_test, y_normal_test),
    verbose=1
)

loss_normal, accuracy_normal = normal_model.evaluate(X_normal_test, y_normal_test, verbose=0)
print(f"Normal Dataset - Test Loss: {loss_normal:.4f}")
print(f"Normal Dataset - Test Accuracy: {accuracy_normal:.4f}")

Melatih dan mengevaluasi model Convolutional Neural Network (CNN) yang telah Anda definisikan sebelumnya, secara spesifik menggunakan dataset setelah LPF.


In [ ]:
input_shape = X_lpf_train.shape[1:]
num_classes = len(np.unique(y_lpf_enc))

print(f"Input shape for CNN (LPF): {input_shape}")
print(f"Number of classes (LPF): {num_classes}")

#  Training and Evaluation untuk data LPF
print("\nTraining dan Evaluasi untuk LPF Dataset")

lpf_model = build_cnn_model(input_shape, num_classes)
lpf_model.summary()

history_lpf = lpf_model.fit(
    X_lpf_train, y_lpf_train,
    epochs=50,
    batch_size=2,
    validation_data=(X_lpf_test, y_lpf_test),
    verbose=1
)

loss_lpf, accuracy_lpf = lpf_model.evaluate(X_lpf_test, y_lpf_test, verbose=0)
print(f"LPF Dataset - Test Loss: {loss_lpf:.4f}")
print(f"LPF Dataset - Test Accuracy: {accuracy_lpf:.4f}")

Melatih dan mengevaluasi model Convolutional Neural Network (CNN) secara spesifik menggunakan dataset setelah BPF.


In [ ]:
input_shape = X_bpf_train.shape[1:]
num_classes = len(np.unique(y_bpf_enc))

print(f"Input shape for CNN (BPF): {input_shape}")
print(f"Number of classes (BPF): {num_classes}")

# Training dan Evaluasi untuk data BPF
print("\nTraining dan Evaluasi Model untuk data BPF")

bpf_model = build_cnn_model(input_shape, num_classes)
bpf_model.summary()

history_bpf = bpf_model.fit(
    X_bpf_train, y_bpf_train,
    epochs=50,
    batch_size=2,
    validation_data=(X_bpf_test, y_bpf_test),
    verbose=1
)

loss_bpf, accuracy_bpf = bpf_model.evaluate(X_bpf_test, y_bpf_test, verbose=0)
print(f"BPF Dataset - Test Loss: {loss_bpf:.4f}")
print(f"BPF Dataset - Test Accuracy: {accuracy_bpf:.4f}")

Memvisualisasikan hasil kinerja model CNN yang telah dilatih pada tiga dataset yang berbeda (Normal, LPF, dan BPF) untuk membandingkan mana yang menghasilkan akurasi tertinggi dan loss terendah.

In [ ]:
import matplotlib.pyplot as plt

# CKumpulkan hasil
results = {
    "Normal": {"accuracy": accuracy_normal, "loss": loss_normal},
    "LPF": {"accuracy": accuracy_lpf, "loss": loss_lpf},
    "BPF": {"accuracy": accuracy_bpf, "loss": loss_bpf}
}

# estrak data untuk plotting
dataset_names = list(results.keys())
accuracies = [results[name]["accuracy"] for name in dataset_names]
losses = [results[name]["loss"] for name in dataset_names]

# Perbandingan Akurasi Pemetaan
plt.figure(figsize=(10, 6))
plt.bar(dataset_names, accuracies, color=['skyblue', 'lightcoral', 'lightgreen'])
plt.xlabel("Dataset Type")
plt.ylabel("Test Accuracy")
plt.title("Perbandingan Test Accuracy Model CNN pada Berbagai Set Data")
plt.ylim(0, 1) # akurasi antara 0 - 1
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Perbandingan Kerugian dalam Grafik
plt.figure(figsize=(10, 6))
plt.bar(dataset_names, losses, color=['skyblue', 'lightcoral', 'lightgreen'])
plt.xlabel("Dataset Type")
plt.ylabel("Test Loss")
plt.title("Perbandingan Test Loss Model CNN pada Berbagai Set Data")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

print("Grafik perbandingan kinerja dihasilkan")